> You are a code generator and reviewer that can only answer with python code.

# Example 1

File: 'example.ts'

```typescript
function greet(name: string) {
  console.log("Hello, " + name);
}
```

**Add the age of the user to the console log.**

In [25]:
with open('example.ts', 'w') as f: f.write("""
function greet(name: string, age : number) {
  console.log(`Hello, ${name}. You are ${age} years old.`);
}
""")

# Example 2

File: '../../src/finishReason.ts'

```typescript
export enum FinishReason {
  length,
  contentFilter,
  stop,
  null,
  cancelled
}
```

**My compiler says that FinishReason "timeout" is missing. Fix the issue.**

In [ ]:
with open('../../src/finishReason.ts', 'w') as f: f.write("""
export enum FinishReason {
  length,
  contentFilter,
  stop,
  null,
  cancelled,
  timeout
}
""")

# Example 3

In [1]:
from pathlib import Path
from typing import List
import re

def print_file_content(file_path: Path):
    content = file_path.read_text()

    # Remove single-line comments
    content = re.sub(r"//.*", "", content)

    # Remove multi-line comments
    content = re.sub(r"/\*[\s\S]*?\*/", "", content)

    print(f"\n\nFile: '{file_path}'")
    print("\n", content, "")

def process_files(file_patterns: List[str]):
    for pattern in file_patterns:
        for file in Path().glob(pattern):
            print_file_content(file)

file_patterns = ["../../src/completion.ts", "../../src/cellUtils.ts", "../../src/tokenUtils.ts"]
process_files(file_patterns)



File: '../../src/completion.ts'

 import axios, { AxiosResponse } from "axios";
import { Configuration, CreateChatCompletionRequest, CreateChatCompletionResponse, OpenAIApi } from "openai";
import { CancellationToken, NotebookCellKind, NotebookEdit, NotebookRange, WorkspaceEdit, window, workspace } from "vscode";
import { appendTextToCell, convertCellsToMessages, insertCell } from "./cellUtils";
import { CompletionType } from "./completionType";
import { addParametersFromMetadata as addNotebookConfigParams, getOpenAIApiKey, getTokenLimit } from "./config";
import { msgs } from "./constants";
import { FinishReason } from "./finishReason";
import { bufferWholeChunks, streamChatCompletion } from "./streamUtils";
import { applyTokenReductions, countTokens, countTotalTokens } from "./tokenUtils";
import { UIProgress, waitForUIDispatch } from "./uiProgress";
import { TiktokenModel } from "@dqbd/tiktoken";

const output = window.createOutputChannel("Notebook ChatCompletion");

async functio

The following code is used in Python to estimate how many tokens the request message format from the OpenAI ChatCompletion message will take on top of the message content.

```python
def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0301"):
    """Returns the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        print("Warning: model not found. Using cl100k_base encoding.")
        encoding = tiktoken.get_encoding("cl100k_base")
    if model == "gpt-3.5-turbo":
        print("Warning: gpt-3.5-turbo may change over time. Returning num tokens assuming gpt-3.5-turbo-0301.")
        return num_tokens_from_messages(messages, model="gpt-3.5-turbo-0301")
    elif model == "gpt-4":
        print("Warning: gpt-4 may change over time. Returning num tokens assuming gpt-4-0314.")
        return num_tokens_from_messages(messages, model="gpt-4-0314")
    elif model == "gpt-3.5-turbo-0301":
        tokens_per_message = 4  # every message follows <|start|>{role/name}\n{content}<|end|>\n
        tokens_per_name = -1  # if there's a name, the role is omitted
    elif model == "gpt-4-0314":
        tokens_per_message = 3
        tokens_per_name = 1
    else:
        raise NotImplementedError(f"""num_tokens_from_messages() is not implemented for model {model}. See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens.""")
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>
    return num_tokens
```

As you can see in the completion.ts from a VSCode extension, I am currently not counting tokens correctly. I am first converting the messages object to JSON and then counting the token of the resulting string, which deviates a lot from the real token count. Please correct the typescript implementation by taken reference from the python example with the num_tokens_from_messages function and apply the changes by overwriting the files like you did in Example 1 and 2.

In [2]:
with open('../../src/completion.ts', 'w') as f: f.write("""
import axios, { AxiosResponse } from "axios";
import { Configuration, CreateChatCompletionRequest, CreateChatCompletionResponse, OpenAIApi } from "openai";
import { CancellationToken, NotebookCellKind, NotebookEdit, NotebookRange, WorkspaceEdit, window, workspace } from "vscode";
import { appendTextToCell, convertCellsToMessages, insertCell } from "./cellUtils";
import { CompletionType } from "./completionType";
import { addParametersFromMetadata as addNotebookConfigParams, getOpenAIApiKey, getTokenLimit } from "./config";
import { msgs } from "./constants";
import { FinishReason } from "./finishReason";
import { bufferWholeChunks, streamChatCompletion } from "./streamUtils";
import { applyTokenReductions, countTokens, countTotalTokens } from "./tokenUtils";
import { UIProgress, waitForUIDispatch } from "./uiProgress";
import { TiktokenModel } from "@dqbd/tiktoken";

const output = window.createOutputChannel("Notebook ChatCompletion");

function numTokensFromMessages(messages: any[], model: string = "gpt-3.5-turbo-0301"): number {
  const encoding = TiktokenModel.encoding_for_model(model);
  let tokensPerMessage = 4;
  let tokensPerName = -1;

  let numTokens = 0;
  for (const message of messages) {
    numTokens += tokensPerMessage;
    for (const key in message) {
      const value = message[key];
      numTokens += encoding.encode(value).length;
      if (key === "name") {
        numTokens += tokensPerName;
      }
    }
  }
  numTokens += 3;
  return numTokens;
}

async function streamResponse(
  response: AxiosResponse<CreateChatCompletionResponse, any>,
  cancelToken: CancellationToken,
  cellIndex: number,
  ck: NotebookCellKind | undefined,
  progress: UIProgress
) {
  const editor = window.activeNotebookEditor!;

  for await (let textToken of bufferWholeChunks(streamChatCompletion(response, cancelToken))) {
    if (Object.values(FinishReason).includes(textToken as FinishReason)) {
      const currentCell = window.activeNotebookEditor!.notebook.cellAt(cellIndex);
      const text = currentCell.document.getText();

      if (!/\S/.test(text)) {
        const edit = new WorkspaceEdit();
        edit.set(currentCell.notebook.uri, [NotebookEdit.deleteCells(new NotebookRange(currentCell.index, currentCell.index + 1))]);
        await workspace.applyEdit(edit);
      }

      return textToken as FinishReason;
    } else {
      output.append(textToken.toString());
    }

    if (typeof textToken !== "string") {
      throw new Error(`Unknown stream result: ${textToken}`);
    }

    if (textToken.includes("```python\n")) {
      ck = NotebookCellKind.Code;

      cellIndex = await insertCell(editor, cellIndex, ck, "python");
      textToken = textToken.replace("```python\n", "");
    } else if (textToken.includes("```") && ck === NotebookCellKind.Code) {
      textToken = textToken.replace("```", "");

      ck = NotebookCellKind.Markup;
      cellIndex = await insertCell(editor, cellIndex, ck);
    }

    if (ck === undefined) {
      cellIndex = await insertCell(editor, cellIndex, NotebookCellKind.Markup);
      ck = NotebookCellKind.Markup;
    }

    await appendTextToCell(editor, cellIndex, textToken);

    progress.report({ increment: 0.5, message: msgs.receivingTokens });
  }

  return FinishReason.length;
}

export async function generateCompletion(
  cellIndex: number,
  completionType: CompletionType,
  progress: UIProgress,
  cancelToken: CancellationToken
): Promise<FinishReason> {
  const e = window.activeNotebookEditor!;
  let messages = await convertCellsToMessages(cellIndex, completionType);
  let ck: NotebookCellKind | undefined = undefined;

  const openaiApiKey = await getOpenAIApiKey();

  if (!openaiApiKey) {
    throw new Error(msgs.apiKeyNotSet);
  }

  const openai = new OpenAIApi(new Configuration({ apiKey: openaiApiKey }));

  const tokenSource = axios.CancelToken.source();
  cancelToken.onCancellationRequested(tokenSource.cancel);

  const nbMetadata = e.notebook.metadata.custom;
  const defaultModel = workspace.getConfiguration().get<string>("notebook-chatcompletion.defaultModel");
  const model: TiktokenModel = nbMetadata?.model ?? defaultModel;
  const temperature = nbMetadata?.temperature ?? 0;
  const limit = getTokenLimit(model);

  progress.report({ message: msgs.calculatingTokens, increment: 1 });
  await waitForUIDispatch();

  const totalTokenCount = numTokensFromMessages(messages, model);

  if (limit !== null && totalTokenCount > limit) {
    const tokenOverflow = totalTokenCount - limit;

    progress.report({ message: msgs.calculatingTokeReductions, increment: 1 });
    await waitForUIDispatch();

    const reducedMessages = await applyTokenReductions(messages, tokenOverflow, limit, model);

    if (!reducedMessages) {
      return FinishReason.cancelled;
    }

    messages = reducedMessages;
  }

  let reqParams: CreateChatCompletionRequest = {
    model: model,
    messages: messages,
    stream: true,
    temperature: temperature,
  };

  if (limit) {
    const reducedTokenCount = numTokensFromMessages(messages, model);
    reqParams.max_tokens = limit - reducedTokenCount;

    if (reqParams.max_tokens < 1) {
      const result = await window.showInformationMessage(
        `The request is estimated to be ${-reqParams.max_tokens} tokens over the limit (including the input) and will likely be rejected from the OpenAI API. Do you still want to proceed?`,
        { modal: true },
        "Yes"
      );
      if (result !== "Yes") {
        return FinishReason.cancelled;
      }
      else{
        
        reqParams.max_tokens = undefined;
      }
    }
  }

  reqParams = addNotebookConfigParams(nbMetadata, reqParams);

  output.appendLine("\n" + JSON.stringify(reqParams, undefined, 2) + "\n");
  progress.report({ increment: 1, message: msgs.sendingRequest });

  const response = await openai.createChatCompletion(reqParams, {
    cancelToken: tokenSource.token,
    responseType: "stream",
  });

  return await streamResponse(response, cancelToken, cellIndex, ck, progress);
}
""")